In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz


In [ ]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fname_dir="/content/blood/"
fname_url="/content/drive/MyDrive/blood/Update_train.csv"
fname=fname_url.split("/")[-1]

In [ ]:
fname_dir="/content/blood/"
!mkdir $fname_dir

In [ ]:
!cp $fname_url $fname_dir


# Download full data set


In [ ]:
fname_url="/content/drive/MyDrive/blood/export/*.csv"
!cp $fname_url $fname_dir

cp: cannot stat '/content/drive/MyDrive/blood/export/*.csv': No such file or directory


In [ ]:
fnames_list=["Exports_ldl_rows_135_full.csv","Exports_ldl_rows_410_full.csv",
             "Export_hgb_rows_965_full.csv","Exports_hgb_rows_2457_full.csv",
             "Exports_hdl_rows_483_full.csv","Exports_hdl_rows_162_full.csv"]
fname_dir="/content/blood/"


In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


1. fájl feldolgozása

In [ ]:
import pandas as pd

In [ ]:
def create_absorbance_columns():
    out=[]
    for i in range(170):
        out.append("absorbance"+str(i))
    return(out)

In [ ]:
def absorbance_converter(stri):
    stri=stri.strip()
    stri=stri.replace("  "," ")
    str_list=stri.split(" ")
    #print(str_list)
    #print(str_list[0], "__")

    if len(str_list) >2: 
        o=[float(i) for i in str_list]
        #print(o[0])
    return o



In [ ]:
def replace_cat(df,category,rule={"low":0,"ok":1,"high":3}):
    df[category].replace(rule,inplace=True)
    return df



In [ ]:
def aggregate_df(df,function="mean",id="donation_id"):
    func=[function]
    cols=list(df.columns[:174])
    df_agg=pd.DataFrame()
    for col in cols:
    
        result=df.groupby(id).agg({col:func})
    
        df_agg[col]=result[col].iloc[:,0].values
    return df_agg

In [ ]:
file_name=fname_dir+fnames_list[5]

In [ ]:

df= pd.read_csv(file_name)
df.tail()

,donation_id,scan_number,absorbance,std,temperature,humidity,hdl_cholesterol_human,hdl_cholesterol_value
9715,1974,56,0.50601782661611 0.51158822870518 0.5147283842...,0.0055,40.75,40.18,low,43.0
9716,1974,57,0.50778758998839 0.51512993898216 0.5143828070...,0.0055,40.82,40.13,low,43.0
9717,1974,58,0.50743548720067 0.5093311024747 0.51146598204...,0.0055,40.87,40.08,low,43.0
9718,1974,59,0.50433959005825 0.5123006489633 0.51491088307...,0.0055,40.94,40.02,low,43.0
9719,1974,60,0.50592106137813 0.50982814612043 0.5126781494...,0.0055,41.00,39.97,low,43.0


In [ ]:
absorbance_columns=create_absorbance_columns()

In [ ]:
df=replace_cat(df,"hdl_cholesterol_human")

In [ ]:
absorbance_list=[]
for i in df.index:
    #print(f"index={i}")
    
    absorbance_list.append(absorbance_converter(df.iloc[i,2]))

In [ ]:
__blood_measure__="cholesterol_hdl"

In [ ]:
ldl1_df= pd.DataFrame(absorbance_list)
ldl1_df.columns=absorbance_columns

ldl1_df["donation_id"]=df.donation_id
ldl1_df["temperature"]=df.temperature
ldl1_df["humidity"]=df.humidity

if __blood_measure__=="cholesterol_ldl":
    ldl1_df["cholesterol_ldl_value"]=df.cholesterol_ldl_value
    ldl1_df["cholesterol_ldl_human"]=df.cholesterol_ldl_human
elif __blood_measure__=="hemoglobin":
    ldl1_df["hemoglobin(hgb)_value"]=df["hemoglobin(hgb)_value"]
    ldl1_df["hemoglobin(hgb)_human"]=df["hemoglobin(hgb)_human"]
elif __blood_measure__=="cholesterol_hdl":
    ldl1_df["hdl_cholesterol_value"]=df.hdl_cholesterol_value
    ldl1_df["hdl_cholesterol_human"]=df.hdl_cholesterol_human


In [ ]:
ldl1_df.describe()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,hdl_cholesterol_value,hdl_cholesterol_human
count,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,...,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000,9720.000000
mean,0.506450,0.508717,0.512355,0.517126,0.523300,0.529539,0.535761,0.541645,0.550366,0.562620,0.575847,0.587535,0.594170,0.597291,0.598597,0.598408,0.597132,0.594805,0.591874,0.588229,0.585164,0.582092,0.578814,0.574884,0.571364,0.568169,0.565667,0.563603,0.561830,0.560365,0.559583,0.559254,0.559517,0.560561,0.562266,0.564657,0.567485,0.570976,0.575006,0.579240,...,1.517847,1.505432,1.493155,1.481511,1.467753,1.456029,1.444512,1.433495,1.422474,1.412844,1.400364,1.391151,1.382585,1.373882,1.364479,1.356945,1.349255,1.341947,1.335310,1.328046,1.322856,1.318467,1.314952,1.312651,1.312082,1.312245,1.312365,1.312297,1.310213,1.305594,1.300079,1.294601,1.291972,1.305154,1.294857,4140.907407,40.883997,37.245149,47.272111,0.691358
std,0.032956,0.032791,0.032745,0.032757,0.032950,0.033119,0.033130,0.033137,0.032979,0.033190,0.033098,0.033028,0.033003,0.033027,0.033158,0.033231,0.033258,0.033287,0.033120,0.032994,0.032852,0.032810,0.032779,0.032781,0.032770,0.032701,0.032654,0.032596,0.032545,0.032487,0.032473,0.032430,0.032421,0.032411,0.032457,0.032485,0.032540,0.032669,0.032755,0.032961,...,0.087314,0.085998,0.084773,0.083635,0.081970,0.080798,0.079770,0.078845,0.078159,0.077059,0.076084,0.075541,0.074845,0.074267,0.073586,0.072990,0.072524,0.072049,0.071361,0.071095,0.070736,0.070607,0.070616,0.070755,0.071213,0.071981,0.073088,0.074612,0.077316,0.081090,0.086452,0.092499,0.097687,0.105109,0.106515,1994.055943,2.967590,7.247857,10.958861,0.787943
min,0.392912,0.394812,0.400689,0.403960,0.408725,0.413678,0.419136,0.426281,0.436464,0.448900,0.461060,0.472540,0.479179,0.481304,0.483719,0.483216,0.481971,0.480212,0.477067,0.473347,0.470453,0.467312,0.464336,0.460893,0.457083,0.453718,0.452251,0.450437,0.448239,0.447033,0.446642,0.446144,0.446081,0.447483,0.448803,0.451550,0.454424,0.457708,0.461049,0.465596,...,1.184005,1.176505,1.165242,1.156129,1.144757,1.137224,1.124663,1.118230,1.112741,1.102217,1.095277,1.086852,1.077

In [ ]:
ldl1_df_agg=aggregate_df(ldl1_df)

In [ ]:
ldl1_df_agg.describe()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,hdl_cholesterol_value
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,...,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000
mean,0.506450,0.508717,0.512355,0.517126,0.523300,0.529539,0.535761,0.541645,0.550366,0.562620,0.575847,0.587535,0.594170,0.597291,0.598597,0.598408,0.597132,0.594805,0.591874,0.588229,0.585164,0.582092,0.578814,0.574884,0.571364,0.568169,0.565667,0.563603,0.561830,0.560365,0.559583,0.559254,0.559517,0.560561,0.562266,0.564657,0.567485,0.570976,0.575006,0.579240,...,1.530670,1.517847,1.505432,1.493155,1.481511,1.467753,1.456029,1.444512,1.433495,1.422474,1.412844,1.400364,1.391151,1.382585,1.373882,1.364479,1.356945,1.349255,1.341947,1.335310,1.328046,1.322856,1.318467,1.314952,1.312651,1.312082,1.312245,1.312365,1.312297,1.310213,1.305594,1.300079,1.294601,1.291972,1.305154,1.294857,4140.907407,40.883997,37.245149,47.272111
std,0.032617,0.032457,0.032438,0.032466,0.032662,0.032845,0.032878,0.032905,0.032767,0.033005,0.032931,0.032875,0.032859,0.032889,0.033026,0.033106,0.033135,0.033164,0.032998,0.032873,0.032733,0.032695,0.032666,0.032675,0.032669,0.032607,0.032565,0.032515,0.032472,0.032422,0.032418,0.032381,0.032382,0.032380,0.032437,0.032472,0.032535,0.032671,0.032765,0.032977,...,0.088491,0.087093,0.085774,0.084566,0.083438,0.081772,0.080606,0.079585,0.078659,0.077989,0.076897,0.075898,0.075365,0.074674,0.074094,0.073392,0.072797,0.072326,0.071835,0.071152,0.070873,0.070485,0.070356,0.070332,0.070459,0.070880,0.071589,0.072640,0.074067,0.076629,0.080149,0.085095,0.090587,0.094878,0.101310,0.099999,2000.136181,2.851457,7.203559,10.992277
min,0.400606,0.403625,0.407266,0.411518,0.416126,0.421518,0.426629,0.433139,0.444069,0.456182,0.469249,0.480239,0.486473,0.489661,0.490777,0.490226,0.488814,0.486476,0.483668,0.479653,0.476146,0.473155,0.469834,0.466424,0.462388,0.459335,0.457297,0.455538,0.453853,0.452486,0.451719,0.451624,0.452026,0.453133,0.454851,0.457279,0.460600,0.463820,0.467916,0.472011,...,1.212829,1.202957,1.193580,1.184336,1.175013,1.163368,1.154944,1.145616,1.137301,1.128859,1.118666,1.110864,1.102883,1.096296,1.089810,1.080864,1.075002,1.068992,1.063488,1.057937,1.052125,1.047685,1.0

 ## Save

In [ ]:
ldl1_df.to_csv("hdl2_full.csv")

In [ ]:
ldl1_df_agg.to_csv("hdl2_agg.csv")

In [ ]:
dest_dir="/content/drive/MyDrive/blood/rework/"
!cp ./*.csv $dest_dir


In [ ]:
#------